In [58]:
import sys
sys.path.insert(0, "../..")
import config as cfg
import gc
import os
from tqdm.notebook import tqdm
from helper import check_path, seed_everything
from collections import defaultdict

In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [60]:
import catboost as cb
import catboost.datasets as cbd
import catboost.utils as cbu

In [61]:
EXPERIMENT_FAMILY_NAME = 'catboost'
EXPERIMENT_NAME = 'deeppavlov'
N_RANDOM_SEEDS = 2

In [62]:
RANDOM_STATE = 77
seed_everything(RANDOM_STATE)

In [63]:
def filter_rare_categories(df: pd.DataFrame) -> pd.DataFrame:
    rare_categories = [12]
    return df.loc[~df[cfg.TARGET].isin(rare_categories)]

In [64]:
train = pd.read_pickle(os.path.join(cfg.PREPROCESSED_DATA_PATH, 'train.pkl')) # .drop(cfg.TEXT_COL, axis=1)  #  cfg.THEME_COL, cfg.TEXT_COL
test = pd.read_pickle(os.path.join(cfg.PREPROCESSED_DATA_PATH, 'test.pkl')) #.drop(cfg.TEXT_COL, axis=1) # cfg.TEXT_COL

CLASSES = np.sort(train[cfg.TARGET].unique()).tolist()

train = filter_rare_categories(train)

In [65]:
emb_names = ['deeppavlov', 'rubert_tiny', 'smaller_LaBSE_15lang']
for emb_name in emb_names:
    train_emb = pd.read_pickle(os.path.join(cfg.DATA_PATH, emb_name, 'train.pkl'))
    test_emb = pd.read_pickle(os.path.join(cfg.DATA_PATH, emb_name, 'test.pkl'))

    train = train.join(train_emb)
    test = test.join(test_emb)

del train_emb, test_emb; gc.collect()

1137

In [66]:
# def rename_columns(df, model_name):
#     df = df.rename(columns={c: f'{model_name}_{c}' for c in df.columns})
#     return df

# keras_model_names = ['labse_emb', 'xlm']
# for keras_model in keras_model_names:
#     train_oof = pd.read_pickle(os.path.join(cfg.OOF_PRED_PATH, 'keras', f'{keras_model}.pkl'))
#     train_oof= rename_columns(train_oof, keras_model)
#     test_pred = pd.read_pickle(os.path.join(cfg.TEST_PRED_PATH, 'keras', f'{keras_model}.pkl'))
#     test_pred = rename_columns(test_pred, keras_model)

#     train = train.join(train_oof)
#     test = test.join(test_pred)

# del train_oof, test_pred; gc.collect()

In [67]:
X_train, y_train = train.drop(cfg.TARGET, axis=1), train[cfg.TARGET]

In [68]:
pred_proba_oof = pd.DataFrame(data=np.zeros(shape=(len(train), len(CLASSES))), index=train.index, columns=CLASSES)
pred_proba_test = pd.DataFrame(data=np.zeros(shape=(len(test), len(CLASSES))), index=test.index, columns=CLASSES)

In [69]:
leak_test = pd.read_pickle(os.path.join(cfg.DATA_PATH, 'test_leak.pkl'))
leak_mask = leak_test.notnull()

In [70]:
N_SPLITS = 5

In [71]:
cv = StratifiedKFold(n_splits=N_SPLITS, random_state=cfg.RANDOM_STATE, shuffle=True)

test_pool = cb.Pool(
        data=test,
        text_features=cfg.TEXT_COLS,
        # cat_features=cfg.CAT_COLS
        )


metrics = defaultdict(list)
fold = 0
nrs = 0
weights = []
for train_idx, val_idx in tqdm(cv.split(X_train, y_train), total=N_SPLITS):

    train_pool = cb.Pool(
        data=X_train.iloc[train_idx], 
        label=y_train.iloc[train_idx],
        text_features=cfg.TEXT_COLS,
        # cat_features=cfg.CAT_COLS
        )

    val_pool = cb.Pool(
        data=X_train.iloc[val_idx], 
        label=y_train.iloc[val_idx],
        text_features=cfg.TEXT_COLS,
        # cat_features=cfg.CAT_COLS
        )
    
    for random_seed in tqdm(range(N_RANDOM_SEEDS), total=N_RANDOM_SEEDS):

        clf = cb.CatBoostClassifier(
            iterations=1000,
            silent=True,
            depth=8,
            # l2_leaf_reg=3,
            eval_metric='AUC',
            classes_count=len(CLASSES),
            random_seed=random_seed,
            early_stopping_rounds=150,
            task_type="GPU",
            devices='0:1',
        )

        clf.fit(train_pool, eval_set=val_pool, plot=True)
        
        model_name = f'{EXPERIMENT_NAME}_fold_{fold}_rs_{random_seed}.cbm'
        model_path = os.path.join(cfg.MODELS_PATH, EXPERIMENT_FAMILY_NAME, EXPERIMENT_NAME)
        check_path(model_path)
        clf.save_model(os.path.join(model_path, model_name))
        
        pred_proba_oof_val = clf.predict_proba(val_pool)[:, ]
        pred_proba_oof_train = clf.predict_proba(train_pool)

        pred_proba_oof.iloc[val_idx, :] += pred_proba_oof_val
        pred_proba_test.iloc[:, :] += clf.predict_proba(test_pool)

        y_train_oof = y_train.iloc[train_idx]
        y_val_oof = y_train.iloc[val_idx]

        train_auc = roc_auc_score(y_train_oof, pred_proba_oof_train , multi_class='ovo', labels=CLASSES)
        val_auc = roc_auc_score(y_val_oof, pred_proba_oof_val , multi_class='ovo', labels=CLASSES)
        metrics['train_auc'].append(train_auc)
        metrics['val_auc'].append(val_auc)
        print('train auc', train_auc, 'val auc', val_auc)

        leak_test_auc_score = roc_auc_score(leak_test.loc[leak_mask], pred_proba_test.loc[leak_mask] / (nrs + 1), multi_class='ovo', labels=CLASSES)
        metrics['leak_test_auc_score'].append(leak_test_auc_score)
        print('leak_test_auc_score', leak_test_auc_score)
        
        del clf; gc.collect()
        nrs += 1
    del train_pool,val_pool; gc.collect() 
        
    fold += 1
pred_proba_oof /= N_RANDOM_SEEDS
pred_proba_test /= (cfg.N_SPLITS * N_RANDOM_SEEDS)

  0%|          | 0/5 [00:00<?, ?it/s]

/home/as/miniconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?it/s]

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Found only 16 unique classes in the data, but have defined 17 classes. Probably something is wrong with data.
Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


...

Found only 16 unique classes in the data, but have defined 17 classes. Probably something is wrong with data.
Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


train auc 0.9999766458980713 val auc 0.8213047386038794
leak_test_auc_score 0.8032954186680399


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Found only 16 unique classes in the data, but have defined 17 classes. Probably something is wrong with data.
Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


train auc 0.9997118107464208 val auc 0.7553573637892919
leak_test_auc_score 0.8041211989980522


In [72]:
oof_auc_score = roc_auc_score(y_train, pred_proba_oof , multi_class='ovo', labels=CLASSES)
print('oof_auc_score', oof_auc_score)

oof_auc_score 0.7916123847374047


In [76]:
pred_proba_test.sum(1)

id
843     1.666667
1422    1.666667
2782    1.666667
2704    1.666667
1       1.666667
          ...   
2619    1.666667
2518    1.666667
2524    1.666667
1577    1.666667
2567    1.666667
Length: 1000, dtype: float64

In [74]:
leak_test_auc_score = roc_auc_score(leak_test.loc[leak_mask], pred_proba_test.loc[leak_mask], multi_class='ovo', labels=CLASSES)
print('leak_test_auc_score', leak_test_auc_score)


ValueError: Target scores need to be probabilities for multiclass roc_auc, i.e. they should sum up to 1.0 over classes

In [ ]:
# 0.8534401490760913

In [ ]:
submission = pd.read_csv(cfg.SAMPLE_SUBMIT_PATH).set_index('id')
assert submission.index.equals(pred_proba_test.index)
submission[cfg.TARGET] = pred_proba_test.idxmax(1)

submission_path = os.path.join(cfg.SUBMISSION_PATH, EXPERIMENT_FAMILY_NAME)
check_path(submission_path)
submission.to_csv(os.path.join(submission_path, f'{EXPERIMENT_NAME}.csv'))

pred_proba_oof_path = os.path.join(cfg.OOF_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_oof_path)
pred_proba_oof.to_pickle(os.path.join(pred_proba_oof_path, f'{EXPERIMENT_NAME}.pkl'))

pred_proba_test_path = os.path.join(cfg.TEST_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_test_path)
pred_proba_test.to_pickle(os.path.join(pred_proba_test_path, f'{EXPERIMENT_NAME}.pkl'))